<a href="https://colab.research.google.com/github/9105507/Trabajo_Drones_2023_2024/blob/main/DQN_Pruebas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train a Deep Q Network with TF-Agents

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/1_dqn_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Introduction


## Setup

If you haven't installed the following dependencies, run:

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet
!pip install tf-keras

In [ ]:
import os
# Keep using keras-2 (tf-keras) rather than keras-3 (keras).
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb
import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

import gym
from gym import spaces

import numpy as np
from tf_agents.environments import py_environment
from tf_agents.specs import array_spec
from tf_agents.environments import tf_py_environment
from tf_agents.trajectories import time_step as ts  # Importar ts

In [ ]:
# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

In [ ]:
tf.version.VERSION

## Hyperparameters

In [ ]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

## Environment

In [ ]:
# env_name = 'CartPole-v0'
# env = suite_gym.load(env_name)

# env.reset()
# PIL.Image.fromarray(env.render())

# print('Observation Spec:')
# print(env.time_step_spec().observation)

# print('Reward Spec:')
# print(env.time_step_spec().reward)

# print('Action Spec:')
# print(env.action_spec())

# train_py_env = suite_gym.load(env_name)
# eval_py_env = suite_gym.load(env_name)

# train_env = tf_py_environment.TFPyEnvironment(train_py_env)
# eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [ ]:
# class CustomEnv(py_environment.PyEnvironment):
#     def __init__(self):
#         super(CustomEnv, self).__init__()

#         # Definir el espacio de observación y el espacio de acción como TensorSpec
#         self._observation_spec = array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=0, maximum=9, name='observation')
#         self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')

#         # Inicializar el estado actual
#         self._state = None

#     def action_spec(self):
#         return self._action_spec

#     def observation_spec(self):
#         return self._observation_spec

#     def _reset(self):
#         # Restablecer el estado inicial aleatorio
#         self._state = np.random.randint(10)
#         return ts.restart(np.array([self._state], dtype=np.int32))

#     def _step(self, action):
#         # Verificar si el estado actual está definido
#         if self._state is None:
#             raise ValueError("El estado actual no está definido. Debe llamar a reset() primero.")

#         # Ejecutar la acción y actualizar el estado
#         self._state += action

#         # Calcular la recompensa
#         reward = 1 if self._state % 2 == 0 else -1

#         # Verificar si se ha alcanzado un estado final
#         done = self._state >= 10

#         # Devolver la observación, la recompensa, si el episodio ha terminado y la información adicional
#         return ts.transition(np.array([self._state], dtype=np.int32), reward=reward, discount=1.0 if done else 0.0)

#     def render(self, mode='human'):
#         # Renderizar el estado actual (opcional)
#         print(f"Estado actual: {self._state}")

#     def close(self):
#         pass

class CardGameEnv(py_environment.PyEnvironment):
  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,), dtype=np.int32, minimum=0, name='observation')
    self._state = 0
    self._episode_ended = False

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = 0
    self._episode_ended = False
    return ts.restart(np.array([self._state], dtype=np.int32))

  def _step(self, action):

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    # Make sure episodes don't go on forever.
    if action == 1:
      self._episode_ended = True
    elif action == 0:
      new_card = np.random.randint(1, 11)
      self._state += new_card
    else:
      raise ValueError('`action` should be 0 or 1.')

    if self._episode_ended or self._state >= 21:
      reward = self._state - 21 if self._state <= 21 else -21
      return ts.termination(np.array([self._state], dtype=np.int32), reward)
    else:
      return ts.transition(
          np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)

# Crear una instancia del entorno personalizado
env = CardGameEnv()

train_py_env = CardGameEnv()
eval_py_env = CardGameEnv()

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## Agent

In [ ]:
fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

## Policies


In [ ]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

example_environment = tf_py_environment.TFPyEnvironment(
    CardGameEnv())

time_step = example_environment.reset()

random_policy.action(time_step)

## Metrics and Evaluation


In [ ]:
def compute_avg_return(environment, policy, num_episodes=10):
    total_return = 0.0
    max_steps_per_episode = 1000  # Definir el límite máximo de pasos por episodio

    for _ in range(num_episodes):
        time_step = environment.reset()
        episode_return = 0.0
        episode_step = 0  # Contador de pasos en el episodio

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
            episode_step += 1

            if episode_step >= max_steps_per_episode:
                break  # Salir del bucle si se supera el límite de pasos

        total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

compute_avg_return(eval_env, random_policy, num_eval_episodes)

## Replay Buffer


In [ ]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

For most agents, `collect_data_spec` is a named tuple called `Trajectory`, containing the specs for observations, actions, rewards, and other items.

In [ ]:
agent.collect_data_spec

In [ ]:
agent.collect_data_spec._fields

## Data Collection

In [ ]:
py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(train_env.reset())

The replay buffer is now a collection of Trajectories.

In [ ]:
# For the curious:
# Uncomment to peel one of these off and inspect it.
# iter(replay_buffer.as_dataset()).next()

The agent needs access to the replay buffer. This is provided by creating an iterable `tf.data.Dataset` pipeline which will feed data to the agent.

Each row of the replay buffer only stores a single observation step. But since the DQN Agent needs both the current and next observation to compute the loss, the dataset pipeline will sample two adjacent rows for each item in the batch (`num_steps=2`).

This dataset is also optimized by running parallel calls and prefetching data.

In [ ]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

dataset

In [ ]:
iterator = iter(dataset)
print(iterator)

In [ ]:
# For the curious:
# Uncomment to see what the dataset iterator is feeding to the agent.
# Compare this representation of replay data
# to the collection of individual trajectories shown earlier.

# iterator.next()

## Training the agent

Two things must happen during the training loop:

-   collect data from the environment
-   use that data to train the agent's neural network(s)

This example also periodicially evaluates the policy and prints the current score.

The following will take ~5 minutes to run.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

# Reset the environment.
time_step = train_py_env.reset()

# Create a driver to collect experience.
collect_driver = py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=collect_steps_per_iteration)

for _ in range(num_iterations):

  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

**DIVERGE, NO FUNCIONA**
**CAMBIAR DE EJEMPLO DE ENTORNO, A OTRO, EL DE LA DUCHA POR EJEMPLO (VÍDEO YOUTUBE)**

# Todo el código junto

In [ ]:
# #ENTORNO
# class ShowerEnv(py_environment.PyEnvironment):
#     def __init__(self):
#         # Actions we can take, down, stay, up
#         self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int32, minimum=0, maximum=2, name='action')
#         # Temperature array
#         self._observation_spec = array_spec.BoundedArraySpec(shape=(1,), dtype=np.float32, minimum=0, maximum=100, name='observation')
#         # Set start temp
#         self._state = 38 + random.randint(-3, 3)
#         # Set shower length
#         self._shower_length = 60
#         # Initialize current time step
#         self._current_time_step = None

#     def action_spec(self):
#         return self._action_spec

#     def observation_spec(self):
#         return self._observation_spec

#     def _reset(self):
#         # Reset shower temperature
#         self._state = 38 + random.randint(-3, 3)
#         # Reset shower time
#         self._shower_length = 60
#         self._current_time_step = time_step.restart(np.array([self._state], dtype=np.float32))
#         return self._current_time_step

#     def _step(self, action):
#         if self._current_time_step is None:
#             return self.reset()

#         # Apply action
#         self._state += action - 1
#         # Reduce shower length by 1 second
#         self._shower_length -= 1

#         # Calculate reward
#         if 37 <= self._state <= 39:
#             reward = 1
#         else:
#             reward = -1

#         # Check if shower is done
#         if self._shower_length <= 0:
#             done = True
#         else:
#             done = False

#         # Apply temperature noise
#         # self._state += random.randint(-1, 1)

#         # Set placeholder for info
#         info = {}

#         self._current_time_step = time_step.transition(
#             np.array([self._state], dtype=np.float32),
#             reward=reward,
#             discount=1.0 if done else 0.0)

#         return self._current_time_step

#     def render(self):
#         # Implement viz
#         pass

# # Crear una instancia del entorno personalizado
# env = ShowerEnv()

# train_py_env = ShowerEnv()
# eval_py_env = ShowerEnv()

# train_env = tf_py_environment.TFPyEnvironment(train_py_env)
# eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

# #AGENTE
# fc_layer_params = (100, 50)
# action_tensor_spec = tensor_spec.from_spec(env.action_spec())
# num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# # Define a helper function to create Dense layers configured with the right
# # activation and kernel initializer.
# def dense_layer(num_units):
#   return tf.keras.layers.Dense(
#       num_units,
#       activation=tf.keras.activations.relu,
#       kernel_initializer=tf.keras.initializers.VarianceScaling(
#           scale=2.0, mode='fan_in', distribution='truncated_normal'))

# # QNetwork consists of a sequence of Dense layers followed by a dense layer
# # with `num_actions` units to generate one q_value per available action as
# # its output.
# dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
# q_values_layer = tf.keras.layers.Dense(
#     num_actions,
#     activation=None,
#     kernel_initializer=tf.keras.initializers.RandomUniform(
#         minval=-0.03, maxval=0.03),
#     bias_initializer=tf.keras.initializers.Constant(-0.2))
# q_net = sequential.Sequential(dense_layers + [q_values_layer])

# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# train_step_counter = tf.Variable(0)

# agent = dqn_agent.DqnAgent(
#     train_env.time_step_spec(),
#     train_env.action_spec(),
#     q_network=q_net,
#     optimizer=optimizer,
#     td_errors_loss_fn=common.element_wise_squared_loss,
#     train_step_counter=train_step_counter)

# agent.initialize()

# #POLICY
# eval_policy = agent.policy
# collect_policy = agent.collect_policy

# random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
#                                                 train_env.action_spec())

# example_environment = tf_py_environment.TFPyEnvironment(
#     ShowerEnv())

# time_step = example_environment.reset()

# random_policy.action(time_step)

# #METRICS
# def compute_avg_return(environment, policy, num_episodes=10):
#     total_return = 0.0
#     max_steps_per_episode = 1000  # Definir el límite máximo de pasos por episodio

#     for _ in range(num_episodes):
#         time_step = environment.reset()
#         episode_return = 0.0
#         episode_step = 0  # Contador de pasos en el episodio

#         while not time_step.is_last():
#             action_step = policy.action(time_step)
#             time_step = environment.step(action_step.action)
#             episode_return += time_step.reward
#             episode_step += 1

#             if episode_step >= max_steps_per_episode:
#                 break  # Salir del bucle si se supera el límite de pasos

#         total_return += episode_return

#     avg_return = total_return / num_episodes
#     return avg_return.numpy()[0]


# # See also the metrics module for standard implementations of different metrics.
# # https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

# compute_avg_return(eval_env, random_policy, num_eval_episodes)

# #REPLAY BUFFER
# table_name = 'uniform_table'
# replay_buffer_signature = tensor_spec.from_spec(
#       agent.collect_data_spec)
# replay_buffer_signature = tensor_spec.add_outer_dim(
#     replay_buffer_signature)

# table = reverb.Table(
#     table_name,
#     max_size=replay_buffer_max_length,
#     sampler=reverb.selectors.Uniform(),
#     remover=reverb.selectors.Fifo(),
#     rate_limiter=reverb.rate_limiters.MinSize(1),
#     signature=replay_buffer_signature)

# reverb_server = reverb.Server([table])

# replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
#     agent.collect_data_spec,
#     table_name=table_name,
#     sequence_length=2,
#     local_server=reverb_server)

# rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
#   replay_buffer.py_client,
#   table_name,
#   sequence_length=2)

# #DATA COLLECTION
# py_driver.PyDriver(
#     env,
#     py_tf_eager_policy.PyTFEagerPolicy(
#       random_policy, use_tf_function=True),
#     [rb_observer],
#     max_steps=initial_collect_steps).run(train_py_env.reset())

# # Dataset generates trajectories with shape [Bx2x...]
# dataset = replay_buffer.as_dataset(
#     num_parallel_calls=3,
#     sample_batch_size=batch_size,
#     num_steps=2).prefetch(3)

# dataset

# iterator = iter(dataset)
# print(iterator)

# #TRAINING
# try:
#   %%time
# except:
#   pass

# # (Optional) Optimize by wrapping some of the code in a graph using TF function.
# agent.train = common.function(agent.train)

# # Reset the train step.
# agent.train_step_counter.assign(0)

# # Evaluate the agent's policy once before training.
# avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
# returns = [avg_return]

# # Reset the environment.
# time_step = train_py_env.reset()

# # Create a driver to collect experience.
# collect_driver = py_driver.PyDriver(
#     env,
#     py_tf_eager_policy.PyTFEagerPolicy(
#       agent.collect_policy, use_tf_function=True),
#     [rb_observer],
#     max_steps=collect_steps_per_iteration)

# for _ in range(num_iterations):

#   # Collect a few steps and save to the replay buffer.
#   time_step, _ = collect_driver.run(time_step)

#   # Sample a batch of data from the buffer and update the agent's network.
#   experience, unused_info = next(iterator)
#   train_loss = agent.train(experience).loss

#   step = agent.train_step_counter.numpy()

#   if step % log_interval == 0:
#     print('step = {0}: loss = {1}'.format(step, train_loss))

#   if step % eval_interval == 0:
#     avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
#     print('step = {0}: Average Return = {1}'.format(step, avg_return))
#     returns.append(avg_return)

In [ ]:
import tensorflow as tf
from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import array_spec
from tf_agents.agents.dqn import dqn_agent
from tf_agents.policies import random_tf_policy
from tf_agents.networks import q_network
from tf_agents.utils import common
import numpy as np
import imageio
import base64
import matplotlib.pyplot as plt
from IPython.display import HTML

# Definir el entorno personalizado
class CardGameEnv(py_environment.PyEnvironment):
    def _init_(self):
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(1,), dtype=np.int32, minimum=0, maximum=10, name='observation')
        self._state = 0
        self._episode_ended = False
        self._current_time_step = None

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self._state = 0
        self._episode_ended = False
        self._current_time_step = ts.restart(np.array([self._state], dtype=np.int32))
        return self._current_time_step

    def _step(self, action):
        if self._episode_ended:
            return self.reset()

        if action == 1:
            self._episode_ended = True
        elif action == 0:
            new_card = np.random.randint(1, 11)
            self._state += new_card
        else:
            raise ValueError('action should be 0 or 1.')

        if self._episode_ended or self._state >= 21:
            reward = 1 if self._state == 21 else -1
            self._current_time_step = ts.termination(np.array([self._state], dtype=np.int32), reward)
        else:
            reward = 0
            self._current_time_step = ts.transition(np.array([self._state], dtype=np.int32), reward=reward)

        return self._current_time_step

    def render(self, mode='human'):
        print(f"State: {self._state}")

# Inicializar el entorno
train_py_env = CardGameEnv()
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_py_env = CardGameEnv()
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

# Definir la red Q
fc_layer_params = (100,)
q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

# Definir el optimizador
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3)

# Definir el contador de pasos de entrenamiento
train_step_counter = tf.Variable(0)

# Inicializar el agente DQN
agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

# Definir las políticas
eval_policy = agent.policy
collect_policy = agent.collect_policy

random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

example_environment = tf_py_environment.TFPyEnvironment(CardGameEnv())

time_step = example_environment.reset()

random_policy.action(time_step)

# Recopilación de datos inicial
initial_collect_steps = 1000
rb_observer = None  # Este debería ser tu observador de buffer de repetición

py_driver.PyDriver(
    train_env,
    py_tf_eager_policy.PyTFEagerPolicy(
        random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(train_env.reset())

## Visualization


### Plots

Use `matplotlib.pyplot` to chart how the policy improved during training.

One iteration of `Cartpole-v0` consists of 200 time steps. The environment gives a reward of `+1` for each step the pole stays up, so the maximum return for one episode is 200. The charts shows the return increasing towards that maximum each time it is evaluated during training. (It may be a little unstable and not increase monotonically each time.)

In [ ]:
#@test {"skip": true}

iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=250)

### Videos

Charts are nice. But more exciting is seeing an agent actually performing a task in an environment.

First, create a function to embed videos in the notebook.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

Now iterate through a few episodes of the Cartpole game with the agent. The underlying Python environment (the one "inside" the TensorFlow environment wrapper) provides a `render()` method, which outputs an image of the environment state. These can be collected into a video.

In [ ]:
def create_policy_eval_video(policy, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = eval_env.reset()
      video.append_data(eval_py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = eval_env.step(action_step.action)
        video.append_data(eval_py_env.render())
  return embed_mp4(filename)

create_policy_eval_video(agent.policy, "trained-agent")

For fun, compare the trained agent (above) to an agent moving randomly. (It does not do as well.)

In [ ]:
create_policy_eval_video(random_policy, "random-agent")